<a href="https://colab.research.google.com/github/mehtasaurav/Drift-Detection-and-Adaptation-VIT-University/blob/main/concept_drift_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: Induce/simulate concept drift in the last part (20%)
# of y_test by modifying some labels (50% approx). Therefore, changing P(y|X))
drift_size = int(y_test.shape[0] * 0.2)
y_test_drift = y_test[-drift_size:]
modify_idx = np.random.rand(*y_test_drift.shape) <= 0.5
y_test_drift[modify_idx] = (y_test_drift[modify_idx] + 1) % len(np.unique(y_test))
y_test[-drift_size:] = y_test_drift

# Define and fit model
pipeline = Pipeline([("scaler", StandardScaler()), ("model", LogisticRegression())])
pipeline.fit(X_train, y_train)

# Detector configuration and instantiation
config = DDMConfig(warning_level=2.0, drift_level=3.0, min_num_instances=30,)
detector = DDM(config=config)

# Simulate data stream (assuming test label available after prediction)
X_test = np.array(X_test)
y_preds = []
y_trues = []
drift_indexes = []
for i, (X, y_true) in enumerate(zip(X_test, y_test)):
    y_pred = pipeline.predict(X.reshape(1, -1))
    y_preds.append(y_pred)
    y_trues.append(y_true)
    error = 1 - int(y_pred == y_true)
    detector.update(value=error)
    status = detector.status
    if status["drift"]:
        print(f"Drift detected at index {i}")
        break

# Evaluation metrics
accuracy = accuracy_score(y_trues, y_preds)
precision = precision_score(y_trues, y_preds, average="weighted")
recall = recall_score(y_trues, y_preds, average="weighted")
f1 = f1_score(y_trues, y_preds, average="weighted")

print(f"Accuracy: {accuracy*100:.3f}"+"%")
print(f"Precision: {precision*100:.3f}"+"%")
print(f"Recall: {recall*100:.3f}"+"%")
print(f"F1 Score: {f1*100:.3f}"+"%")

# Plot accuracy vs. data samples
n_samples = len(y_test)
x = np.arange(n_samples)
accs = np.zeros(n_samples)
for i in range(n_samples):
    accs[i] = accuracy_score(y_trues[: i + 1], y_preds[: i + 1])
plt.plot(x, accs)
for drift_index in drift_indexes:
    plt.axvline(x=drift_index, linestyle="--", color="r")
plt.title("Frouros Model - DDM")
plt.xlabel("Data Samples")
plt.ylabel("Accuracy")
plt.show()